In [6]:
import os
import pandas as pd
import multiprocessing
import torch
from transformers import pipeline
import joblib


# Disable Hugging Face tokenizers parallelism to avoid multiprocessing conflicts
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Set multiprocessing start method for macOS
multiprocessing.set_start_method("spawn", force=True)

# Check if MPS (Metal Performance Shaders) is available for acceleration
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Load FinBERT sentiment analysis pipeline with MPS acceleration
pipe = pipeline(
    "text-classification", model="ProsusAI/finbert", device=0 if device == "mps" else -1
)


def split_text(text, chunk_size=250, overlap=50):
    """Splits long text into overlapping chunks to fit FinBERT's token limit."""
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunks.append(" ".join(words[i : i + chunk_size]))
    return chunks


def get_sentiment_score(text):
    """Return the sentiment score (-1 to 1) using ProsusAI/FinBERT, handling long text."""
    chunks = split_text(text)  # Get chunks
    total_words = len(text.split())  # Original total word count
    sentiment_scores = []

    for i, chunk in enumerate(chunks):
        try:
            if len(chunk.split()) > 512:
                chunk = " ".join(
                    chunk.split()[:512]
                )  # Ensure chunk does not exceed model limit
            result = pipe(chunk)[0]  # Run chunk through FinBERT model
            label = result["label"]
            score = result["score"]
            word_count = len(chunk.split())

            if label == "positive":
                sentiment_scores.append(score * word_count)
            elif label == "negative":
                sentiment_scores.append(-score * word_count)
            else:
                sentiment_scores.append(0)

        except Exception as e:
            print(f"Error processing chunk {i+1}: {e}")

    if total_words == 0:
        return 0, total_words  # Avoid division by zero

    overall_sentiment = (
        sum(sentiment_scores) / total_words
    )  # Weighted average sentiment
    return overall_sentiment, total_words


def get_weighted_sentiment(speakers):
    """Computes the overall weighted sentiment score from multiple speakers."""
    total_words = 0
    weighted_score_sum = 0

    for speaker_key, speaker_data in speakers.items():
        speaker_name = speaker_data.get("name", speaker_key)
        text = speaker_data["content"]

        score, word_count = get_sentiment_score(text)

        # Accumulate total words and weighted score sum
        total_words += word_count
        weighted_score_sum += score * word_count

    # Compute final weighted sentiment score for the entire dataset
    if total_words > 0:
        return weighted_score_sum / total_words

    return None  # If no valid data


def calculate_earnings_surprise(actual, estimated):
    """Calculate earnings surprise percentage."""
    if estimated == 0:
        return None  # Avoid division by zero
    return (actual - estimated) / estimated


def pipeline(actual, estimated, size, transcript):
    
    earnings_surprise = calculate_earnings_surprise(actual, estimated)
    
    tone = get_weighted_sentiment(transcript)
    
    # Load the model
    rf_model = joblib.load("random_forest_model.pkl")

    # New sample for prediction (must match training format)
    new_data = pd.DataFrame([[earnings_surprise, size, tone]], columns=["Earnings_Surprise", "Firm_Size", "method_2"])

    # Predict category
    predicted_category = rf_model.predict(new_data)
    
    # Get confidence scores for each class
    confidence_scores = rf_model.predict_proba(new_data)

    return predicted_category[0], confidence_scores[0]

if __name__ == "__main__":
    # Sample input formatted with speaker keys
    data = {
        "speaker1": {
            "name": "A - Matthew Jeremy Fassler",
            "content": ":The costs are a little bit higher, but the increases are abating as we go through the year. We had big increases in Q1. Those increases diminish as we go through the year. Consequently, the headwind from PT diminishes and flips to a favorable factor for OR as you move to the back half. :I'll take the first part of that. First of all, I think I alluded to this in my prepared remarks. We saw weight per day \u2013 or rather, weight per shipment move higher and length of haul growth moderate a bit. So that dampened reported yields a bit relative to underlying pricing trends. We are currently tracking double digits in yield quarter-to-date. So you're seeing our yield track much closer to price increases on contract renewals. And Mario will take the second part on new contracts. It's Matt. I'll speak about the impact that the divestiture of intermodal had on our guidance for the second \u2013 for the rest of the year. And hopefully, that will help you get an answer to your question..   So if you look at the delta from the guidance that we issued at the end of the fourth quarter entering the year and the guidance that we just gave you today, we had about $60 million of adjusted EBITDA modeled for intermodal for quarters 2 through 4. That's obviously out. The guidance change reflects that coming out plus the first quarter beat, which versus the midpoint of the range was about at $38 million, and at the midpoint about $12 million of additional EBITDA. So, an intrinsic raise for the rest of the business for quarters 2 through 4. Obviously, the increase in EPS range reflects all those factors as well as the lower interest expense, reflecting the debt paydown from the proceeds of the intermodal transaction. :I don't think the difference between intermodal and the rest of the business in terms of adjusted EBITDA margin is sufficiently different to really move the needle for segment-level margins. If you think about the consumer to services, obviously we're in the goods business, so we're moving both consumer goods and industrial goods. Obviously, the industrial economy is still on its way back. Mario spoke about the opportunity for loosening of supply chains to \u2013 as we see more raw goods and unfinished goods make their way into the \u2013 make their way into our customer supply chains. You're going to see better revenue momentum from industrial customers, and there have been signs of that continuously in LTL.   The consumer has obviously had a couple of very good years. Consumer has very strong balance sheet and is in very strong shape.   And if you think about our brokerage business, where load growth was up 23%, consumer customers \u2013 or rather consumer companies comprise the majority of our customer base and revenue in that business. So that's some indication of what we're seeing from the consumer."
        },
        "speaker2": {
            "name": "Drew Wilkerson",
            "content": "Thanks, Mario.   North American truck brokerage had another very strong quarter. We continue to do what we do best, outpace the industry on volume growth, operate at a strong margin and use our experience and technology to make sure our customers' freight gets where it needs to be.   In the first quarter, our loads were up year-over-year by 23%, driving a 38% increase in revenue. It was our sixth consecutive quarter of load growth over 20%. And margin dollars were up year-over-year by 21%. That's more than double the increase in the fourth quarter. Gross margin per load increased sequentially from the fourth quarter, and our margin percentage was a very healthy 16%.   So a strong start of the year, and we continued with the same strong trends in April with both volume and margin percentage. We built our model as a growth engine that's working exactly as we intended.   There are five compelling advantages specific to our platform. First, 68% of our business is contract based, and about 75% of that contract revenue is locked up on an annual basis. This gives us good visibility into the future revenue performance.   Second, we have many long-term customer relationships with blue-chip market leaders across diverse verticals. These customers think of us as a strategic partner. Our top 10 brokerage customers have an average tenure with us of 15 years, and our top 20 customers have a tenure of 13 years.   Third, we're continuing to expand our pool of independent carriers who provide our customers with truck capacity. This massive capacity is at the heart of our value proposition. As of March, we have relationships with 88,000 carriers in North America and access to more than 1.5 million trucks.   Fourth is our XPO Connect technology, which continues to attract customers and carriers to our business. Our first-mover advantage with brokerage technology is one of the main drivers behind our rapid growth.   XPO Connect lets us capitalize on two interrelated secular trends that work in our favor. More shippers are outsourcing truckload transportation to brokers, and increasingly, these shippers want digital brokerage capabilities. At the same time, more carriers are realizing that we can give them access to thousands of loads on a daily basis.   Carriers and customers love to do business with us because we have a state-of-the-art digital platform. In the first quarter, the number of registered carriers on XPO Connect year-over-year was up 39%, registered customers were up 41% and weekly carrier usage was up 59%. We also surpassed 700,000 cumulative downloads of our mobile app. And 74% of our loads were created or covered digitally in the quarter, which is up 4 points sequentially. That number is already trending higher in the second quarter.   The fifth compelling advantage is our exceptional truck brokerage management team, all of whom will be joining the spin-off. It's a privilege to work with these great operators and technologists since the early days of XPO. Together, we've built a best-in-class brokerage platform that delivers outsized results and puts us in a strong position to spin off as a pure play. As a new company, we'll manage about $7.5 billion of transportation spend and continue to innovate the business to grow it from there.   From day 1, it will be easier for investors to appreciate our flexible, asset-light business model, our economic resilience and our very high return on invested capital. We're excited to take this business to the next level as a separate public company when the spin-off is complete.   With that, I'll turn it over to the operator, and we'll go to Q&A.   Question-and-Answer Session"
        },
        "speaker3": {
            "name": "A - Brad Jacobs",
            "content": ":I'll start with the last part, Scott. The European process is going well. It's a vibrant process. And to date, the Russia-Ukraine thing has not affected it. I think a sale is. :Well, we discussed my 10b5-1 plan at length in the last earnings call. I don't have anything more to add to that.   In terms of my future with the company, I do plan to be with XPO for a very long time. I'm non-Executive Chairman of GXO. I'll be non-Executive Chairman of the new spin. I'd like to remain Executive Chairman of XPO for as long as investors will have me. And I'll remain CEO until someone comes along who I think is better than me. Well, our search and the management lineup is still ongoing. We don't have anything new to add to that.   In terms of timing, as soon as it makes sense. We're not going to rush it. We're not going to delay it. As soon as it make sense, then we'll do it.   In terms of the other question you had about internal versus external, we're considering all possibilities. So long story short, we're staying flexible, as we always are. :Not really, no. We're going full speed ahead with the spin because regardless of what happens in the macro or in economy, the rationale for the spin still stands. The rationale for the spin is, when we have separate companies that are pure-plays and the management team is focused on one thing and one thing alone, they're going to do better. They're just going to do a better job. They'll be more focused, they'll be more fit for purpose.   And from an investor point of view, investors will be able to evaluate those companies on a stand-alone basis and a more comparable basis to the peers, and there'll be a wider universe of investors who only want an LTL company or only want a truck brokerage company. So no, we're going full speed ahead on the spin.   End of Q&A"
        },
        "speaker4": {
            "name": "A - Mario Harik",
            "content": ":Yes, I'll take this on the linehaul contractual rates. So, we just completed our yearly bid for our third-party linehaul, where we outsource linehaul miles to third parties. Here last week is when we had the new rates come in. And contractual rates have actually gone up slightly versus where they used to be, but nowhere near on the level of increases that we've seen, obviously, last year and through Q1, as well, on a year-on-year basis. So, rates are moderating, but the contractual rates are still up. And it was a competitive bid, obviously, across multiple carriers there. But this is what we're seeing on the linehaul outsourcing side with LTL. :Yes. When we look at the OR improvement in LTL, looking at the second quarter and moving forward, it's really driven by three categories. One is pricing, one is volume, and one is operational excellence and costs.   On the pricing side, we continue to see very robust pricing. So we ended the first quarter with a full year improvement of 9% year-on-year. And heading into the second quarter, here, we crossed into double-digit territory.   On the volume side, for the second quarter, we expect to be down low- to mid-single digit and then accelerate in the back half of the year.   And then finally on the cost side, the third component is around the cost headwinds last year, such as purchased transportation, would end up being a much lower headwind as we head through the year and starting here with the second quarter in the month of May.   And also, our network is much more efficient. So now that our network fluidity has improved substantially, also overall our cost footprint is down, which leads to the OR improvement of more than 400 basis points in the second quarter. :And Scott, on the pricing environment in LTL, it continues to be very strong. When we look at the first quarter for us, we \u2013 our contract renewals accelerated to 11%, and our yield in the second quarter accelerated from 9% in the first quarter and crossed double digits so far quarter to date. So, it remains a very strong environment. :Thanks, Chris. So first of all, Q2 tonnage. So we expect the tonnage to be down low to mid-single digits, as I just mentioned. And April was within that range. And \u2013 however, when it comes to revenue, when we look so far in April, our revenue growth per day in the month of April has accelerated versus the Q1 revenue growth per day.   Now when we look at the tonnage momentum as a whole, we still expect tonnage momentum to build through the course of the year, because we have great sales momentum and the team is very energized. That team has driven, here in the month of April, the best win rate we've had in sales over the past 4 years. We have dozens of multimillion-dollar opportunities that have either closed in the month of April or early May or are about to close in the month of May \u2013 including a new top 10 customer. So when we think about the momentum building through the course of the year, we expect to build that momentum in the back half.   And on the balance, it's still a very healthy environment, we expect \u2013 and that's why we expect more than 400 basis points of OR improvement from Q1 going into Q2, and a full year of at least 100 basis points of OR improvement.   Now when we look at the cost side of it, so obviously a big component of our cost is purchased transportation, and these costs will be \u2013 are starting to normalize now as we finalized the bid for linehaul this past week, and the new rates are into effect. So that's going to be an improvement in the back half of the year.   Similarly, when you think about our dock costs or our P&D costs, when we're running a much more efficient network, a lot of these metrics are trending up into the right in terms of higher efficiency of dock labor and P&D labor as well.   So when we head into the back half of the year, these would be some \u2013 and also reducing rehandling costs as well. And all of these would lead to the back half of the year having a lot of these cost pressures abate on a year-on-year basis. Now obviously, there's still inflationary costs with wages and other portions. But all of these would \u2013 again, the pressure on cost is going to abate in the back half. :Yes. Let me try to cover all of them. So why \u2013 the first one is on tonnage. So it's fair to say that there are crosswinds in the freight environment.   Now I'll tell you, we frequently engage with a group of salespeople in the company and we get their feedback on what's happening, what feedback they're hearing from customers. And we just had the last review here yesterday. And we're still hearing a lot of optimism about demand from customers. But we're also hearing that the lockdowns in China limited some of the inflows of goods into the country. And some of the shortages, either the chip shortages or the raw material shortages, are also impacting some of our industrial customers. Now we feel overall as these dynamics abate, and they will, these will become natural tailwinds for us in the back half of the year.   And as I mentioned earlier, what we're excited about is our momentum from a sales perspective, where we have an energized sales team that is driving great results as we head into the back half.   And now in terms of the adding capacity in the door plan, so obviously we don't look at adding doors as being a short-term lever. LTL is a very attractive industry that has great vitality through the cycles. Today, we're a top 3 pure-play provider when the spin is completed. And our company-wide ROIC is 38%, and LTL has a higher ROIC than that. So when we think about adding doors, equipment and headcount, it's supporting the long-term vision of the network, where we are going to gain market share and get back to growing our top line, both from a volume perspective and obviously from capitalizing on the yields as well as the industry. :Yes. Based on what we're getting from customers, it continues to be a very robust pricing environment. If you think of our industry, you have the top 10 players in the business that today manage roughly 76% of the $51 billion LTL industry. And when you think about those dynamics, the pricing environment is still firm in LTL. And as I said earlier, from a sales perspective, we just landed a top 10 customer, and we continue to have great momentum in terms of opportunities in the pipeline as well. And pricing continues to be robust. :Thanks, Allison. I'll talk on the pricing platform.   So we're very excited about the new pricing platform we launched in Q1 with continuous enhancements through the course of the year, because it's driven by advanced analytics and it allows us \u2013 around customer shipment data. So we analyze customer data more efficiently. And it really helps our pricing experts. It reduces manual tasks and manual data processing by as much as 80%, where it enables the pricing experts to spend more time negotiating with the customer and supporting our sales team to do so. It also enables us to analyze RFP data, of RFPs we submitted in the past, which feed into lead generation tools.   Similarly, we're also \u2013 this coming quarter, we're launching new capabilities with a proprietary costing model that allows us to improve how we look at costing in our network across the board. And we also improved dynamic pricing, where we can adjust pricing based on market conditions and customer demand as well.   Now when you look at the overall results, obviously we accelerated our yield from 9% in the first quarter to double digits. We crossed the double-digit mark in the second quarter, with 11% contract renewals in the first quarter. Now it's tough to be able to get to how much of that was the work that the technology is doing versus the people are doing, but technology is definitely having a big contribution as part of that."
        },
        "speaker5": {
            "name": "Ravi Tulsyan",
            "content": "Thank you, Brad, and good morning, everyone. Today, I will discuss our first quarter results, our balance sheet and liquidity and our outlook for the balance of 2022.   For the first quarter, we delivered strong year-over-year growth in revenue, adjusted EBITDA and adjusted diluted EPS. Revenue in the quarter was a record $3.5 billion, up 16% year-over-year. The net impact of fuel prices and FX accounted for 2 points. Organic revenue growth for the quarter was 14%.   We grew adjusted EBITDA by 15% to a Q1 record of $321 million. Adjusting for gains from real estate sales, the year-over-year growth in adjusted EBITDA was 25%. This reflects particularly strong growth in our brokerage and other services segment. Looking at a 2-year stack, adjusted EBITDA was up 55%.   For the quarter, our adjusted EBITDA margin was 9.2%. Excluding gains from real estate sales, this was an improvement of 60 basis points year-over-year.   Operating conditions in the quarter were favorable, and the pricing environment stayed firm. This was partially offset by inflationary pressures on labor and purchased transportation.   Corporate costs in the quarter were down 19% year-over-year as we continue to optimize our corporate cost structure following the spin-off of GXO.   Our interest expense during the quarter was $37 million compared to $65 million in the year-ago period. This reflects the paydown of approximately $3 billion of debt last year.   The effective tax rate for adjusted EPS during the quarter was 23%.   Our adjusted earnings per diluted share for the quarter was $1.25, which was up from $0.79 a year ago, an increase of 58%. This increase was primarily driven by higher adjusted EBITDA and lower interest expense.   We generated $200 million of cash flow from continuing operations, spent $137 million on growth CapEx and received $3 million of proceeds from asset sales. Growth CapEx was up $63 million year-over-year with the majority of the additional spend going towards equipment purchases for the North American LTL business. As a result, our free cash flow was $66 million, which was above our expectation. This includes the impact of $15 million of cash outflows related to transaction costs that were not contemplated in our free cash flow guidance.   We are making significant progress on our strategy to create 2 pure-play transportation powerhouses, and we remain on track to complete the spin-off of our tech-enabled brokerage platform in Q4 of this year. As part of our strategic plan, we took an important step last quarter when we completed the sale of our intermodal business for cash proceeds of $710 million, which represented a multiple of approximately 10x 2021 EBITDA.   Including the proceeds from the sale of the intermodal business, we ended the quarter with $1 billion of cash on the balance sheet. This cash, combined with available debt capacity under committed borrowing facilities, give us $2 billion of liquidity at quarter end.   We had no borrowings outstanding under our ABL facility. After quarter end, we repaid $630 million of 2025 notes. This was another significant step in our plan to reduce our debt and deleverage our balance sheet.   Our net leverage at quarter end was 2x adjusted EBITDA. We are ahead of schedule on our deleveraging plan, and we now expect to be below 2x leverage before year-end.   In light of our strong first quarter results and ongoing earnings visibility, we updated our full year guidance after market close yesterday. Our new full year guidance for adjusted EBITDA is $1.35 billion to $1.39 billion. The update reflects our first quarter outperformance, the sale of the intermodal business and our strong outlook for the remainder of the year. The real estate assumptions we gave you in February remain the same.   For the second quarter, we expect our adjusted EBITDA to be $360 million to $370 million. Pro forma for the intermodal sale, the midpoint of our second quarter EBITDA guidance implies a year-over-year growth rate of 15%.   We have raised our outlook for full year adjusted EPS to a range of $5.20 to $5.60. This increase reflects our new EBITDA guidance and the reduction in interest expense resulting from our paydown of debt. The midpoint of our adjusted EPS guide implies year-over-year growth of 26%.   On the cash flow front, our outlook for full year free cash flow remains $400 million to $450 million. As a reminder, our outlook excludes all transaction-related cash outflows.   Our full year guidance for depreciation and amortization expense is approximately $385 million, down from $400 million, reflecting the sale of the intermodal business.   We expect interest expense of $150 million to $160 million, down from $170 million to $180 million previously.   There is no change to our previous guidance for CapEx and the tax rate.   In conclusion, we are continuing to execute on our strategic plan, and we remain excited about our prospects for the balance of 2022.   I will now turn things over to Matt."
        },
        "speaker6": {
            "name": "A - Drew Wilkerson",
            "content": "This is Drew. On the second part of your question, we're in the early innings of continuing to go out and take market share within our truck brokerage team. Obviously, we've grown volume by 20% for 6 consecutive quarters. So overall, this is \u2013 the trucking industry is a $400 billion for-hire trucking market. Of that, brokers have about $88 billion. And we're \u2013 our brokerage is roughly $3 billion. So when you look at it, we've only got about 3% share of the market. Our technology has given us first-mover advantage and is allowing us to continue to go out and take share. Yes. So thank you for the question.   So on the cycle, you look at it right now and rates are up on a year-over-year basis overall. As we start heading into the back half of the year, there are a few indicators that show you that capacity could actually tighten as the year goes on. First, you've got beverage season, which is coming in, and that's going to create about 100, 120 days where your major beverage distributors pick up their volume during that time. You also have DOT checkpoint week that comes in over the summer, which typically causes a little bit of chaos in the market. And the third thing is you have a lot of built-up demand that's sitting in the ports of China that's going to come over and hit the U.S. ports. So all of that tells you that it could tighten up a little bit as we head into the back half of the year.   On our digital orders, it's something that you're going to continue to see go up into the right. I don't think that there's a limit to what we can do. Obviously, we've invested in technology from day 1. It's been an 11-year investment for us, and it's something that has allowed us to become sticky with both the customers and the carriers that we're working with. So we expect that to continue to go. And it's one of the things that \u2013 our technology, as you hit on, is one of the things that allows us to come in and have best-in-class EBITDA margins, and we'll continue to see gains off of that. Thanks for the question, Christopher. Our digital orders trend in line with our overall orders as far as margin percentage goes. It does allow us to leverage the SG&A leverage, though, Christopher. So the EBITDA margins can be higher. :Yes, that's absolutely possible. Yes, absolutely. Thank you for the question. On big picture, we've got a model that is proven as working. So we want to continue to execute the model. It has allowed us to go out and to grow volume by over 20% for the last 6 quarters, and it's allowed us for the last 8 years to outperform the industry revenue growth by more than 3x. So we're going to continue to execute on the model.   But there's three things that I'm focused on right now. The first is continuing to build out an exceptional management team that complements the great operators and technologists that we have in the business.   The second piece is to go out and continue to delight our customers and service our customers extremely well and go out and continue to outperform and take market share.   And the third is to continue to evolve XPO Connect. Our goal for XPO Connect is to have that to be the go-to tool for shippers as they are using that to make their transportation decisions, whether it's what mode of transportation that they're using, whether it's what day of the week they're shipping something or if they should consolidate something. We want that to be their go-to tool.   On the third \u2013 on your last point on headcount, we are continuing to add headcount as we go forward. And as you look at it, over a 5-year trend, volume still significantly outpaces head count that we've added. :We're constantly adding the bells and whistles. That's something that we continue to do. We do that on a biweekly basis. And we're excited to share more with you as we get closer to the Investor Day and the spinoff."
        },
        "speaker7": {
            "name": "Mario Harik",
            "content": "Thanks, Matt, and good morning, everyone. As Brad said, LTL is a very different business than it was 6 months ago. Our network is much more fluid and balanced. We're realizing significantly stronger service metrics in key areas such as on-site transit and freight handling. The sharp rebound has brought our network efficiency back to pre-pandemic levels.   Our investments in capacity are on track, and we're rolling out new, proprietary technology to improve pricing and drive further productivity. Most importantly, there is widespread excitement across the LTL team around the many new initiatives we've launched to optimize the network.   I'll give you an example. This month, we kicked off a national initiative to further improve the quality of our trailer loading and on-time delivery. We're also engaging with customers on best practices in how they package their freight. When we announced this initiative, it triggered a tidal wave of enthusiasm in the field. In just a few weeks, we've seen great momentum building in the organization.   Our goal is to create a world-class LTL carrier that delights our customers and our investors. It won't happen tomorrow, but the commitment is there, and we're already moving in that direction. I wanted to give you the backdrop first because the transformation we're undertaking in LTL helped us beat our guide for adjusted operating ratio in the first quarter.   In February, we guided to 200 basis points of year-over-year degradation, reflecting our continuous progress in moving the operating ratio back toward year-over-year improvement. In reality, we did better than the guide. Our Q1 degradation was just 140 basis points. This gave us an adjusted operating ratio ex real estate of 85.7% for the quarter. We expect to inflect to year-over-year improvement in adjusted operating ratio at some point later this quarter, as we said we would. And we're on target for a year-over-year improvement of more than 100 basis points in adjusted operating ratio ex real estate for the full year.   We plan to drive hundreds of additional basis points of OR improvement in the coming years to get the ratio well into the 70s. And importantly, the underlying trends are favorable. We reported record first quarter revenue of $1.1 billion, which is 15% higher than last year. Our year-over-year yield improvement was a first quarter record at 9%. Tonnage was down slightly year-over-year within the rates we told you to expect.   When you look one step deeper, the sequential trends are also positive. Growth in revenue per shipment accelerated every month in the quarter. Our yield also accelerated throughout the quarter with the strongest gains coming in March. We're operating in a very robust pricing environment.   Our tonnage in the quarter was up 4% sequentially, which is better than typical seasonality. Usually, our tonnage is flat from Q4 to Q1.   And as I mentioned, the momentum in our operating ratio is building ahead of plan. We expect our second quarter adjusted operating ratio to improve by more than 400 basis points from Q1 to Q2. So the business is very much on track.   Now I want to cover some of the first quarter actions we took to expand our capacity and deploy technology. These are the two main areas where we're investing in growth.   We recently opened new terminals in California and Georgia. These sites bring our net new doors to 345 since October, against a target of 900 net new doors added by year-end 2023. So we're more than 1/3 of the way there.   The San Bernardino terminal will serve growing demand from manufacturing and retail in Southern California. And the Atlanta terminal expands our capacity in one of the largest LTL regions in the South. It also gives us a larger gateway into Florida. These 2 sites will be growth levers for us in 2023 and going forward.   We're also making great progress in expanding our fleet of trailers. In the first quarter, our manufacturing facility in Arkansas produced more trailers than any other quarter in our history. We also opened new fleet maintenance shops in Florida, Ohio, Nevada and New York, which will help us manage maintenance costs. We continue to budget for growth CapEx at 8% to 9% of revenue this year. And, again, we're on plan.   On the technology front, recent developments include the ongoing application of our proprietary pricing platform, which is driven by advanced automated analytics around customer shipment data. This means our pricing experts are now able to focus more time on analyzing account performance, and we can negotiate with better results. In Q1, our price increases on contract renewals accelerated to 11%.   In April, we brought on board David Phalen as Senior Vice President of Pricing for LTL. David's career includes decades of success in optimizing pricing for transportation sectors, including the airline industry, where he specialized in pricing and yield management.   In other developments, we launched automated billing for accessorials to make sure we capture those revenue dollars. And we rolled out a new digital dashboard with self-service tools to enhance the customer experience. This quarter, we'll be introducing proprietary cost models to enhance visibility into cost management levers as well as new piece-level tracking functionality for our network.   So to sum it up, our comprehensive growth and optimization strategy for LTL comes down to one goal: we're creating a world-class LTL carrier. And we have every confidence that we'll succeed at this goal, just as we succeeded in dramatically increasing LTL returns over the first 6 years we owned the business.   We continue to expect to generate at least $1 billion of adjusted EBITDA in LTL in 2022. This is nearly triple the adjusted EBITDA generated by the LTL network in 2015, when we acquired it from Con-way. What you're seeing from us in 2022 is that we're full steam ahead, laser focused on efficiency and superb customer service. And our momentum will continue to grow, because we have the support of our people, who are some of the best LTL operators in the industry.   Now Drew will cover truck brokerage. Drew?"
        },
        "speaker8": {
            "name": "Brad Jacobs",
            "content": "Good morning, everybody. Thanks for joining our call. With me today in Greenwich are Ravi Tulsyan, our CFO; Matt Fassler, our Chief Strategy Officer; Mario Harik, our CIO and Acting President of LTL; and Drew Wilkerson, President of North American transportation.   As you saw, we delivered a quarter of record results with solid beats across revenue, net income, adjusted EBITDA and adjusted EPS, including our eighth straight quarterly beat on adjusted EBITDA. We grew revenue by 16% over last year's first quarter, generating the highest revenue of any quarter in our history. We also had first quarter records for net income, adjusted EBITDA and adjusted EPS, which was up year-over-year by 58%. To reflect our momentum, we raised our full year financial outlook by more than the first quarter beat. We're now guiding, at the midpoint, to adjusted diluted EPS growth of 26%.   Both of our core North American businesses, LTL and truck brokerage, delivered double-digit revenue growth in the quarter. Our LTL network today is a very different business than it was just 6 months ago. Many of the network improvements we're driving are ahead of plan.   We expect our adjusted operating ratio to improve sequentially by more than 400 basis points for the second quarter. And we continue to expect our adjusted operating ratio, excluding real estate gains, to inflect to year-over-year improvement later in the quarter, and for the full year to be at least 100 basis points better than 2021. Our goal is to create a world-class LTL carrier that delights our customers and our shareholders.   Turning to North American truck brokerage. We're continuing to fire on all cylinders there. Our volume growth exceeded 20% for the sixth consecutive quarter. This best-in-class brokerage business is continuing to take share and do it profitably. First quarter margin dollars were up 21% year-over-year and rose 5% sequentially, outperforming typical seasonality in large part due to the effectiveness of our digital platform, XPO Connect.   I'm pleased that we have a brokerage veteran lined up to lead the spin-off: Drew Wilkerson. Drew will be CEO, and he's been the main architect of our brokerage growth since joining us in 2012, most recently as President of North American transportation. He'll have a long runway to grow the business as a pure-play brokerage company. The spin-off process is on track, as is the planned divestiture of our European operations, which had an excellent first quarter.   And finally, we're pleased with the deleveraging we achieved in the first quarter. In 3 months, we brought our net leverage ratio down from 2.7x to 2x, which is at the top edge of our target range.   So in sum, we produced an excellent quarter and raised our outlook. We have multiple company-specific avenues for value creation, including the spin-off of our tech-enabled brokerage platform, the ongoing transformation of our LTL business, the divestiture of our European operations and our continuing deleveraging.   Now I'll ask Ravi to cover our results and our balance sheet. Ravi? :well, thank you, operator. And I'd like to leave everyone with the following points.   Number one, we're executing on multiple avenues to create outsized shareholder value. In LTL, we're ahead of plan and we have enormous momentum. If you look at April, the growth in revenue per day in April exceeds the first quarter's growth.   Year-over-year OR comparisons should flip favorable in the back half of Q2, and we firmly expect to generate more than 100 basis points of improvement in OR for the full year.   In truck brokerage, we continue to perform at best-in-class levels. In the first quarter, we grew loads on a year-over-year basis at 23%. And April was our best month ever for margin dollars in truck brokerage. And we expect to continue to significantly outperform the market.   On the balance sheet side, I'm very pleased that we brought leverage down from 2.7 to 2.0 in the first quarter, and we remain on track to reduce leverage below 2x by the end of this year, which is ahead of plan.   On the spin, we're progressing nicely, and we expect to turn 1 great company to 2 great companies in the fourth quarter.   So with that, I'd like to thank everyone. We look forward to seeing you at the upcoming conferences. Have a great day."
        },
        "speaker9": {
            "name": "Matthew Jeremy Fassler",
            "content": "Thanks, Ravi. I'll review our first quarter operating results, starting with our North American LTL segment.   We grew LTL revenue by 15% year-over-year to $1.1 billion, the highest revenue of any quarter in our history. Excluding fuel, we grew revenue by 9% year-over-year. We had a 0.8% decline in tonnage per day, which represented 4.1 percentage points of acceleration from our fourth quarter growth rate as our network flow improved. The 4% increase in the level of weight per day from the fourth quarter nicely outpaced typical seasonality. Yield, excluding fuel, increased 9% year-over-year. Our weight per shipment growth of 2% increased from flat in the fourth quarter, and our length of haul increased 0.2% versus a 1% increase in the fourth quarter. Adjusting for these factors, underlying pricing trends were stronger.   The LTL pricing environment remains firm, and we're driving yield with our own company-specific pricing initiatives. Our yield growth is even stronger this quarter to date.   Our LTL adjusted operating ratio was 85.7%. That's 140 basis points higher than the first quarter a year ago and significantly better than the 200 basis point year-over-year increase we guided to in February. Both numbers exclude gains from real estate, and note that, consistent with our guidance, we booked no such gains in the first quarter. And sequentially, our adjusted OR improved by 180 basis points from the fourth quarter, which was notably better than our typical seasonality.   The single biggest driver of the year-over-year increase in our adjusted operating ratio was the higher cost of purchased transportation. Third-party linehaul miles as a proportion of total linehaul miles increased in the quarter to 24.5% from 23.9% a year ago. This aligned with our plan to use more third-party linehaul in the near term as volume recovered prior to our onboarding new capacity.   In addition, most of our purchased transportation is subject to contractual pricing. Headwinds from this pricing are starting to abate as we cycle big truckload price increases from a year ago, and as new linehaul contracts cycle in.   Those are the highlights for North American LTL.   In our brokerage and other services segment, we grew revenue by 17% to a record $2.4 billion and adjusted EBITDA by 31% to a record $164 million. Adjusted EBITDA margin for this segment expanded by 60 basis points to 6.7% from 6.1% a year ago.   The largest revenue and profit driver in this segment is our North American truck brokerage business, which had another outstanding quarter. This will be the core business of our planned spin-off later this year.   We increased our brokerage loads per day by 23% versus a year ago and up 52% from 2 years ago. First quarter truck brokerage revenue rose 38% year-over-year. Margin dollars rose 21% against another tough comp and nearly tripled from the first quarter of 2020. On a sequential basis, margin dollars in the first quarter were 5% higher than in Q4.   Our truck brokerage growth reflects our strong execution in a dynamic market. Drew will speak more about the specific drivers in a minute.   Organic revenue in Europe grew 5%, accelerating by 2 percentage points from the fourth quarter. We saw acceleration in organic revenue growth in both the U.K. and in France, our two largest European geographies, and we're pleased with our resilience in the context of uncertainty in the European environment. We expect to continue to build on our position as a leading provider of truck brokerage and LTL in the U.K., France and Iberia within our broader pan-European platform.   We're grateful for some external recognition we received during the quarter. General Motors named us Supplier of the Year for the fourth straight year. In addition, we were recently given VETS Indexes employer status for our strong record of hiring members of the military community.   Last week, we published our fourth annual sustainability report, which details our progress on ESG initiatives and our materiality index for 2021. You can download this report online.   Finally, I want to mention that we plan to hold separate investor events later this year for both our brokerage SpinCo and our LTL RemainCo prior to completing the spin. We'll use those events to give you a deep dive into our longer-term vision and key financial targets for each stand-alone company.   Now I'll turn it over to Mario for his comments on North American LTL."
        }
    }

    predicted_category, confidence_scores = pipeline(1.25, 0.94, 6089250000, data)
    print(f"Predicted Category: {predicted_category}, Confidence: {max(confidence_scores) * 100:.2f}%")




Device set to use mps:0


Predicted Category: Bullish, Confidence: 100.00%
